In [ ]:
def webCrawler(mywebpage_url, max_ebooks):
    # Request to get all the content from the seed webpage.
    mywebpage_html=requests.get(mywebpage_url)
    
    # Parse the HTML content using beautifulsoup
    mywebpage_parsed = bs4.BeautifulSoup(mywebpage_html.content,'html.parser')
    #print(mywebpage_parsed)
    
    # During this whole process until the for loop we need to take a look into the developer mode of the 
    #  given webpage in order to fetch the desired part of the webpage, and fetch it through our following commands.
    
    # Finds all the "ol" type features in the body of the parsed webpage.
    webpage_body_elements = mywebpage_parsed.find('body').find_all('ol') 
    #print(webpage_body_elements)

    # Our webpage has 6 types of rankings, from which the 5th one (4 index notation) is our top 100 ebooks last 30 days.
    # We fetch that one.
    hundred_ebooks_thirty_days = webpage_body_elements[4]
    
    # Finds all the type "a" lines where the information about each books is stored (title, address extension, classification 
    # Nº...)
    ebooks_links = hundred_ebooks_thirty_days.find_all('a')
    
    for i in range(0, max_ebooks): # Takes the index values until the maximum size of the length set in the function call.
        
        # FIRST
        each_book_link = ebooks_links[i] # Goes over all links in our list.
        book_title = each_book_link.contents[0] # We see that the first content is always the title, we fetch it directly.
        book_href = each_book_link['href'] 
        # "href" gives us the extension we need to attach to the parent link in order to find
        # our desired location in the webpage.
        
        # SECOND: Creating the new webpage where the different reading options are located, here we find the link for the txt 
        # format link.
        new_URL = url + book_href 
        new_webpage_html = requests.get(new_URL) # We newly repeat the process with our new link, we rwquest the content.
        newpage_Parsed = bs4.BeautifulSoup(new_webpage_html.content,'html.parser') # Parse in order to remove the html content.
        
        print("Downloading...")
        print('Printing the ebook:', book_title)

        webpage_body_elements = newpage_Parsed.find('body') # Find the elements in the body.
        # Here we encounter the situation in which there are two types of defined "a" type content. We define both in order
        # to consider all the possible cases that may appear in each book's txt file link.
        text_plain_anchor = webpage_body_elements.find_all('a', type="text/plain")
        text_plain_charset_anchor = webpage_body_elements.find_all('a', type="text/plain; charset=utf-8")
        
        # We define the conditions to choose the type of text anchor.
        if(len(text_plain_anchor) == 0): # If the initial one is empty, we exclude it. 
            print(text_plain_charset_anchor)
            text_plain_anchor = text_plain_charset_anchor
        else: # ELse, we consider it.
            print(text_plain_anchor)
          
        # THIRD:
        link_extension = text_plain_anchor[0]['href'] # Again, we get the link extension.
        pathURL = url + link_extension # attach it to the initial parent link.
        print(pathURL)
        
        # FOURTH: Open the web file, set the name and location where the downloaded file will be stored.
        with open('eLibrary/' + book_title.replace(":", "") + '.txt', 'wb') as file:   
            response = requests.get(pathURL) # get the content from the link.
            file.write(response.content) # Download it's content into our created file.
    

: 

In [ ]:
# WEB CRAWLER FUNCTION CALL
max_ebooks = 20
mywebpage_url = 'https://www.gutenberg.org/browse/scores/top#books-last30'

webCrawler(mywebpage_url, max_ebooks)

In [ ]:
# gettextlist FUNCTION DEFINITION #
# PARAMS: directory_path (str) - Path of the working directory were the ebooks are downloaded.

def gettextlist(directory_path):
    directory_textfiles=[] # List of .txt files
    directory_nontextfiles=[] # List of non .txt files
    directory_nonfiles=[] # List of non file features.
    
    # Process each item in the directory
    directory_contents=os.listdir(directory_path) # Getter of all the content inside the directory
    for contentitem in directory_contents:
        temp_fullpath=os.path.join(directory_path, contentitem)
        
        # Non-files (e.g. subdirectories) are stored separately
        if os.path.isfile(temp_fullpath)==0:
            directory_nonfiles.append(contentitem)
        else:
            # Is this a non-text file (not ending in .txt)?
            if temp_fullpath.find('.txt')==-1:
                directory_nontextfiles.append(contentitem)
            else:
                # This is a text file
                directory_textfiles.append(contentitem)
    
    return(directory_textfiles,directory_nontextfiles,directory_nonfiles)

In [ ]:
# Fetching all the .txt files from our directory #
# PARAMS: directory_path (str) - Brings the directory path of our working directory in our machine.

def filecrawler(directory_path):
    # Store filenames read and their text content
    num_files_read = 0 # Flag variable to keep control of the number of files processed.
    crawled_booknames = [] # For titles
    crawled_ebooks = [] # For txt files.
    
    # Gets the text files, non text files and non file features from our directory path.
    directory_contentlists = gettextlist(directory_path) 
    
    # In this basic crawled we just process text files and do not handle subdirectories
    directory_textfiles = directory_contentlists[0]
    
    # For loop: goes over the content of all the text files of our selected directory.
    for contentitem in directory_textfiles:
        #print('Reading file:')
        #print(contentitem)
        
        # Open the file and read its contents (In our case the ebooks)
        temp_fullpath = os.path.join(directory_path, contentitem)
        temp_file = open(temp_fullpath,'r',encoding='utf-8',errors='ignore')
        temp_text = temp_file.read()
        temp_file.close()
        
        # Fetches the header and footer messages and stores their index inside the ebook (Index of ALL the text)
        # NOTE: It takes the first index number, not the range it covers.
        headerIndex1 = temp_text.find('*** START OF THE PROJECT GUTENBERG EBOOK')
        headerIndex2 = temp_text.find('*** START OF THIS PROJECT GUTENBERG EBOOK')
        headerIndex3 = temp_text.find('***START OF THE PROJECT GUTENBERG EBOOK')
        
        if headerIndex1 != -1: # If not equal to 'not existing'(=-1)
            actualHeaderIndex = headerIndex1
        elif headerIndex2 != -1:
            actualHeaderIndex = headerIndex2
        else:
            actualHeaderIndex = headerIndex3
        
        # We consider the next line after the header index we identified in our set of conditions.
        actualHeaderIndex = temp_text.find("\n", actualHeaderIndex)
        
        # Same logic for the footer.
        actualFooterIndex1 = temp_text.find('*** END OF THE PROJECT GUTENBERG EBOOK')
        actualFooterIndex2 = temp_text.find('*** END OF THIS PROJECT GUTENBERG EBOOK')
        actualFooterIndex3 = temp_text.find('***END OF THE PROJECT GUTENBERG EBOOK')
        
        if actualFooterIndex1 != -1:
            actualFooterIndex = actualFooterIndex1
        elif actualFooterIndex2 != -1:
            actualFooterIndex = actualFooterIndex2
        else:
            actualFooterIndex = actualFooterIndex3
        
        print("Header Index: ",  actualHeaderIndex, "Footer index: ", actualFooterIndex)
        
        # Store the read filename, that is, the name of the book.
        crawled_booknames.append(contentitem)
        # Only takes the part of the book between the header and footer indexes.
        crawled_ebooks.append(temp_text[actualHeaderIndex : actualFooterIndex]) 
        num_files_read = num_files_read+1
        
    return(crawled_booknames,crawled_ebooks)

In [ ]:
mycrawled_filenames_and_texts = filecrawler(mydirectory_path)
mycrawled_filenames = mycrawled_filenames_and_texts[0] # Names all the .txt files
mycrawled_texts = mycrawled_filenames_and_texts[1] # Contents of all the text files, that is, the ebook.